# **T5 Model fine tuning for CNN/DailyMail**

In [ ]:
!pip install transformers datasets rouge-score nltk bert-score

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import nltk
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bert_score import score as bert_score
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
dataset = load_dataset('cnn_dailymail', '3.0.0')


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
train_dataset = dataset['train'].select(range(10000))
test_dataset = dataset['test']

In [ ]:
model_name = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
def tokenize_function(batch):
    inputs = tokenizer(batch["article"], max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(batch["highlights"], max_length=142, truncation=True, padding="max_length", return_tensors="pt")
    batch["input_ids"] = inputs["input_ids"]
    batch["attention_mask"] = inputs["attention_mask"]
    batch["labels"] = labels["input_ids"]
    return batch

#tokenizing the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights", "id"])
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights", "id"])


NameError: name 'train_dataset' is not defined

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)


In [ ]:
trainer.train()
torch.save(model.state_dict(), "t5_finetuned_cnn.pt")


Epoch,Training Loss,Validation Loss
1,0.727100,1.109521
2,0.619200,1.112234
3,0.544700,1.147954


In [ ]:
'''model = T5ForConditionalGeneration.from_pretrained(model_name)
model.load_state_dict(torch.load("t5_finetuned_cnn.pt"))
model.to(device)
model.eval()'''

In [ ]:
print("Evaluating the model on the test set...")

# Initializing metrics
rouge =load_metric('rouge', trust_remote_code=True)
meteor= load_metric('meteor', trust_remote_code=True)

reference_summaries =[]
generated_summaries =[]

model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for i in tqdm(range(0, len(test_dataset), training_args.per_device_eval_batch_size), desc="Evaluating Test Set"):
    batch = test_dataset.select(range(i, min(i + training_args.per_device_eval_batch_size, len(test_dataset))))
    inputs = {
        "input_ids": torch.tensor(batch["input_ids"]).to(device),
        "attention_mask": torch.tensor(batch["attention_mask"]).to(device)
    }
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=142, early_stopping=True)
    generated_batch = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    references = [tokenizer.decode(l, skip_special_tokens=True, clean_up_tokenization_spaces=False) for l in batch["labels"]]

    # Storing reference and generated summaries
    reference_summaries.extend([nltk.word_tokenize(ref.lower()) for ref in references])
    generated_summaries.extend([nltk.word_tokenize(gen.lower()) for gen in generated_batch])

    # updating ROUGE and METEOR metrics
    rouge.add_batch(predictions=generated_batch, references=references)
    meteor.add_batch(predictions=generated_batch, references=references)
# BLEU Score
bleu_score = corpus_bleu([[ref] for ref in reference_summaries], generated_summaries, smoothing_function=SmoothingFunction().method7)
# ROUGE Score
rouge_result = rouge.compute()
# METEOR Score
meteor_result = meteor.compute()

#cosine Similarity-TF-IDF
vectorizer= TfidfVectorizer().fit(references + generated_batch)
reference_vectors= vectorizer.transform(references)
generated_vectors= vectorizer.transform(generated_batch)
cosine_scores= cosine_similarity(reference_vectors, generated_vectors).diagonal().mean()
#BERTScore
P, R, F1 = bert_score(generated_batch, references, lang='en', verbose=True)
bert_score_f1 = F1.mean().item()

#results
results = {
    "BLEU": bleu_score,
    "ROUGE-1": rouge_result['rouge1'].mid.fmeasure,
    "ROUGE-2": rouge_result['rouge2'].mid.fmeasure,
    "ROUGE-L": rouge_result['rougeL'].mid.fmeasure,
    "METEOR": meteor_result['meteor'],
    "Cosine Similarity": cosine_scores,
    "BERTScore": bert_score_f1
}

Evaluating the model on the test set...


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
Evaluating Test Set: 100%|██████████| 1437/1437 [1:07:30<00:00,  2.82s/it]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.13 seconds, 15.44 sentences/sec


In [ ]:
df = pd.DataFrame([results])
print(df)

       BLEU  ROUGE-1   ROUGE-2  ROUGE-L    METEOR  Cosine Similarity  \
0  0.215366   0.3966  0.179858  0.27886  0.334168           0.381001   

   BERTScore  
0   0.872028  


**VERSION 2 FINE TUNING**

In [ ]:
from transformers import EarlyStoppingCallback, T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

#T5 model reloaded to start fine-tuning from the original base model
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
#defining the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4, #learning rate is reduced
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=5,#increasing the number of epochs
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",
    load_best_model_at_end=True,
)

# Initializing Trainer with early stopping callback
#if no improvement after 2 evaluations
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

# Savesaving the fine-tuned model's state_dict as a .pt file
torch.save(model.state_dict(), "t5_finetuned_cnn_v2.pt")



Epoch,Training Loss,Validation Loss
1,0.688200,1.167084
2,0.646400,1.148991
3,0.614700,1.142294
4,0.573300,1.146732
5,0.565600,1.153465


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model.load_state_dict(torch.load("t5_finetuned_cnn_v2.pt"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
reference_summaries = []
generated_summaries = []

for i in tqdm(range(0, len(test_dataset), training_args.per_device_eval_batch_size), desc="Evaluating Test Set"):
    batch =test_dataset.select(range(i, min(i + training_args.per_device_eval_batch_size, len(test_dataset))))
    inputs ={
        "input_ids": torch.tensor(batch["input_ids"]).to(device),
        "attention_mask": torch.tensor(batch["attention_mask"]).to(device)
    }
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=142, early_stopping=True)
    generated_batch = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    references = [tokenizer.decode(l, skip_special_tokens=True, clean_up_tokenization_spaces=False) for l in batch["labels"]]
    reference_summaries.extend([nltk.word_tokenize(ref.lower()) for ref in references])
    generated_summaries.extend([nltk.word_tokenize(gen.lower()) for gen in generated_batch])
    rouge.add_batch(predictions=generated_batch, references=references)
    meteor.add_batch(predictions=generated_batch, references=references)

# BLEU score
bleu_score = corpus_bleu([[ref] for ref in reference_summaries], generated_summaries, smoothing_function=SmoothingFunction().method7)
#rougr Score
rouge_result = rouge.compute()
#METEOR score
meteor_result = meteor.compute()
#cosinesimilarity-TF-IDF
vectorizer = TfidfVectorizer().fit(references + generated_batch)
reference_vectors = vectorizer.transform(references)
generated_vectors = vectorizer.transform(generated_batch)
cosine_scores = cosine_similarity(reference_vectors, generated_vectors).diagonal().mean()
#BERTScore
P, R, F1 = bert_score(generated_batch, references, lang='en', verbose=True)
bert_score_f1 = F1.mean().item()

#results
results = {
    "BLEU": bleu_score,
    "ROUGE-1": rouge_result['rouge1'].mid.fmeasure,
    "ROUGE-2": rouge_result['rouge2'].mid.fmeasure,
    "ROUGE-L": rouge_result['rougeL'].mid.fmeasure,
    "METEOR": meteor_result['meteor'],
    "Cosine Similarity": cosine_scores,
    "BERTScore": bert_score_f1
}

Evaluating Test Set: 100%|██████████| 1437/1437 [1:11:41<00:00,  2.99s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.07 seconds, 29.93 sentences/sec


In [ ]:
df = pd.DataFrame([results])
print(df)

       BLEU   ROUGE-1   ROUGE-2   ROUGE-L    METEOR  Cosine Similarity  \
0  0.259492  0.404341  0.186791  0.285176  0.347221           0.545989   

   BERTScore  
0    0.90092  


In [ ]:
import os

#output directory
output_dir = "./results"
#list all files in the output directory
checkpoints =[f for f in os.listdir(output_dir) if f.startswith('checkpoint')]

if checkpoints:
    print("Checkpoints found:")
    for checkpoint in checkpoints:
        print(checkpoint)
else:
    print("No checkpoints found.")

Checkpoints found:
checkpoint-3750


Verifying if the model has been loaded from the checkpoint

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
checkpoint_path = "./results/checkpoint-3750"

#loading the model from the checkpoint
model =T5ForConditionalGeneration.from_pretrained(checkpoint_path)

tokenizer =T5Tokenizer.from_pretrained("t5-base")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

#setting the model to evaluation mode
model.eval()

print("Model loaded from checkpoint:", checkpoint_path)

#initializing metrics
rouge =load_metric('rouge', trust_remote_code=True)
meteor =load_metric('meteor', trust_remote_code=True)

reference_summaries =[]
generated_summaries = []

for i in tqdm(range(0, len(test_dataset), training_args.per_device_eval_batch_size), desc="Evaluating Test Set"):
    batch = test_dataset.select(range(i, min(i + training_args.per_device_eval_batch_size, len(test_dataset))))
    inputs = {
        "input_ids": torch.tensor(batch["input_ids"]).to(device),
        "attention_mask": torch.tensor(batch["attention_mask"]).to(device)
    }
    summary_ids= model.generate(inputs["input_ids"], num_beams=4, max_length=142, early_stopping=True)
    generated_batch= [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    references =[tokenizer.decode(l, skip_special_tokens=True, clean_up_tokenization_spaces=False) for l in batch["labels"]]
    reference_summaries.extend([nltk.word_tokenize(ref.lower()) for ref in references])
    generated_summaries.extend([nltk.word_tokenize(gen.lower()) for gen in generated_batch])

    rouge.add_batch(predictions=generated_batch, references=references)
    meteor.add_batch(predictions=generated_batch, references=references)

#BLEUScore
bleu_score = corpus_bleu([[ref] for ref in reference_summaries], generated_summaries, smoothing_function=SmoothingFunction().method7)
#rouge-score
rouge_result =rouge.compute()
#METEORscore
meteor_result= meteor.compute()
#cosinesimilarity
vectorizer= TfidfVectorizer().fit(references + generated_batch)
reference_vectors= vectorizer.transform(references)
generated_vectors= vectorizer.transform(generated_batch)
cosine_scores = cosine_similarity(reference_vectors, generated_vectors).diagonal().mean()
# BERTScore
P, R, F1 = bert_score(generated_batch, references, lang='en', verbose=True)
bert_score_f1 = F1.mean().item()

#results
results = {
    "BLEU": bleu_score,
    "ROUGE-1": rouge_result['rouge1'].mid.fmeasure,
    "ROUGE-2": rouge_result['rouge2'].mid.fmeasure,
    "ROUGE-L": rouge_result['rougeL'].mid.fmeasure,
    "METEOR": meteor_result['meteor'],
    "Cosine Similarity": cosine_scores,
    "BERTScore": bert_score_f1
}
df = pd.DataFrame([results])
print(df)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model loaded from checkpoint: ./results/checkpoint-3750


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Evaluating Test Set: 100%|██████████| 1437/1437 [1:11:40<00:00,  2.99s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.07 seconds, 29.09 sentences/sec
       BLEU   ROUGE-1   ROUGE-2   ROUGE-L    METEOR  Cosine Similarity  \
0  0.259492  0.404341  0.186791  0.285176  0.347221           0.545989   

   BERTScore  
0    0.90092  


In [ ]:
#getting check the metrics after rtainingg
best_model_path = trainer.state.best_model_checkpoint
print(f"Best model checkpoint path: {best_model_path}")

#printing the best evaluation metric
best_eval_metric = trainer.state.best_metric
print(f"Best evaluation metric (e.g., validation loss): {best_eval_metric}")
print("Training Arguments used:")
print(trainer.args)
#other arguments
print(f"Learning Rate: {trainer.args.learning_rate}")
print(f"Batch Size (train): {trainer.args.per_device_train_batch_size}")
print(f"Batch Size (eval): {trainer.args.per_device_eval_batch_size}")
print(f"Number of Epochs: {trainer.args.num_train_epochs}")
print(f"Weight Decay: {trainer.args.weight_decay}")


Best model checkpoint path: ./results/checkpoint-3750
Best evaluation metric (e.g., validation loss): 1.1422942876815796
Training Arguments used:
TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
e